# 项目分析：


- 蛇 Snake：是一个独立的事物
- 食物 Food：是一个独立的事物
- 世界 World：也是，但世界负责显示

In [3]:
import random
import queue
import threading
import time
from tkinter import *

SC_WIDTH  = 400
SC_HEIGHT = 300
STEP = 10

# 随机出现在画面的某个地方
# 被吃掉后，销毁自己，蛇的分数增加
class Food():
    def __init__(self, queue):
        self.queue = queue
        self.new_food()
        self.position = []
        
    def new_food(self):
        x = random.randrange(50, SC_WIDTH-50, STEP)
        y = random.randrange(50, SC_HEIGHT-50, STEP)
        self.position = [x,y]
        self.queue.put({'Food':self.position})
        



# 能移动，由键盘上下左右键控制
# 每次移动，都需要重新计算蛇头的位置
# 检测游戏是否 Gameover
class Snake(threading.Thread):
    
    def __init__(self, world, queue):
        super().__init__()
        
        self.world = world
        self.queue = queue
        self.snakePosPoints = [(150, 100), (140,100), (130,100), (120,100)]
        self.direction = "Up"
        
        self.points_earned = 0 #游戏分数
        self.food = Food(self.queue)

        
        self.start()
    
    def run(self):
        if self.world.isGameover:
            self._delete()
        while not self.world.isGameover:
            self.queue.put({"Move":self.snakePosPoints})
            time.sleep(0.5)
            self.move()

            
    def move(self):
        # 重新获取新一格的蛇头位置
        newPosPoint = self.calNewPos()
        
        if self.food.position == newPosPoint:
            print('xx')
            self.points_earned += 1 #得分加1
            self.queue.put({"points_earned": self.points_earned})
            self.food.new_food() #就得食物被吃掉，产生新的食物
            

            
            
        else: 
            # 弹出蛇尾一格
            self.snakePosPoints.pop(0)
            # 检查是否撞墙
            self.checkGameover(newPosPoint)
            # 增加新的一格
            self.snakePosPoints.append(newPosPoint)
    
    
    def calNewPos(self): 
        last_x, last_y = self.snakePosPoints[-1]
        
        if self.direction == "Up":
            newPosPoint = last_x, last_y - 10
        if self.direction == "Down":
            newPosPoint = last_x, last_y + 10
        if self.direction == "Left":
            newPosPoint = last_x - 10, last_y
        if self.direction == "Right":
            newPosPoint = last_x + 10, last_y
            
        return newPosPoint

    
    def key_pressed(self, event):
        # keysym是按键名称
        self.direction = event.keysym
        print(self.direction)

        
    def checkGameover(self, newpos):
        x,y = newpos[0], newpos[1]
        if not 0 < x < SC_WIDTH or not 0 < y < SC_HEIGHT or newpos in self.snakePosPoints:
            self.queue.put({'GameOver': True})






# 游戏舞台，Tk的子类，绘制所有元素
class World(Tk):
    def __init__(self):
        # 父类的构造函数
        super().__init__()
        
        q = queue.Queue()

        self.queue = q
        self.isGameover = False
        
        # 绘制舞台
        self.canvas = Canvas(self, width=SC_WIDTH, height=SC_HEIGHT, bg='blue')
        self.canvas.pack()
        
        self.snake = self.canvas.create_line((0,0),(0,0),fill='#FFCC4C',width=STEP)
        self.food = self.canvas.create_rectangle(0,0,0,0,fill='#FFCC4C', outline='#FFCC4C')
        
        self.points_earned = self.canvas.create_text(SC_WIDTH-100, 30, fill='white', text='SCORE: 0')

        self.queue_handler()
        
    def queue_handler(self):
        try:
            while True:
                task = self.queue.get(block=False)
                #print(task)
                
                if task.get("GameOver"):
                    self.gameOver()
                elif task.get("Move"):
                    points = [x for point in task['Move'] for x in point]
                    self.canvas.coords(self.snake, *points)                    
                    
                elif task.get("Food"):                    
                    x,y = task['Food'][0],task['Food'][1]
                    points = [x-5,y-5,x+5,y+5]
                    self.canvas.coords(self.food, points)

        
        # 爆出队列空异常
        except queue.Empty:
            if not self.isGameover:
                self.canvas.after(100, self.queue_handler)
                

    def restart(self):
        self.destroy()
        self.__init__()
        
    def gameOver(self):
        self.isGameover = True
        self.canvas.create_text(SC_WIDTH/2, SC_HEIGHT/2, fill='white',text="GameOver")
        
        qb = Button(self,text="Quit", command=self.destroy)
        rb = Button(self,text="Again", command=self.restart)
        qb.pack()
        rb.pack()

        
        
if __name__ == "__main__":
    
    
    
    world = World()
    snake = Snake(world, world.queue)

    # 绑定右键，上下键    
    world.bind('<Left>', snake.key_pressed)
    world.bind('<Right>', snake.key_pressed)
    world.bind('<Up>', snake.key_pressed)
    world.bind('<Down>', snake.key_pressed)
    
    world.mainloop()